In [ ]:
# generate a test excel by removing 3/4 of the rows
import pandas as pd

# file_path = '../Data/Gaze_data/WaterStreet_DayNight 14951.d.xlsx'
# gaze_df = pd.read_excel(file_path, engine='openpyxl')
# rows_to_keep = int(gaze_df.shape[0] / 4)
# gaze_df = gaze_df.head(rows_to_keep)
# gaze_df.to_excel('../Data/Gaze_data_test/r_WaterStreet_DayNight 14951.d.xlsx', index=False)

In [ ]:

def find_unmatched_folders(xlsx_dir, target_dir):
    # List to hold the names of xlsx files without a matching folder
    unmatched = []

    # List all files in the xlsx directory
    for filename in os.listdir(xlsx_dir):
        if filename.endswith('.xlsx'):
            match_d = re.search(r'(\d+\.d)(?=\.xlsx$)', filename)
            match_n = re.search(r'(\d+\.n)(?=\.xlsx$)', filename)
            if match_d or match_n:
                # Extract the base part (numeric ID)
                base_part = match_d.group(1) if match_d else match_n.group(1)
                suffix = "day" if match_d else "night"

                # Now check for a matching folder in target_dir
                # We want a folder that ends with the base_part + suffix
                expected_suffix = f"{base_part[:-1]}{suffix}"
                folder_found = False

                for folder_name in os.listdir(target_dir):
                    if folder_name.endswith(expected_suffix):
                        folder_found = True
                        break

                if not folder_found:
                    unmatched.append(filename)
    
    return unmatched

xlsx_dir = '../Data/Gaze_data'
target_dir = '../Data/Video_data'

unmatched_files = find_unmatched_folders(xlsx_dir, target_dir)
if unmatched_files:
    print("Files without a matching folder:", len(unmatched_files))
    for file in unmatched_files:
        print(file)
else:
    print("All files have a matching folder.")


In [ ]:
# separate files in xlsx_dir into day and night folders
import os
import re

xlsx_dir = '../Data/Gaze_data'
day_list = []
night_list = []

for filename in os.listdir(xlsx_dir):
    if filename.endswith('.xlsx'):
        match_d = re.search(r'(\d+\.d)(?=\.xlsx$)', filename)
        match_n = re.search(r'(\d+\.n)(?=\.xlsx$)', filename)
        if match_d:
            day_list.append(filename)
        elif match_n:
            night_list.append(filename)

print("Day files:", len(day_list))
print("Night files:", len(night_list))
print(day_list)
print(night_list)

In [22]:
# read the xlsx files and extract the data
import pandas as pd
import numpy as np
import cv2

frame_dir = '../Data/train'
os.makedirs(frame_dir, exist_ok=True)

# Size of the rectangle to draw for each gaze point
rect_width, rect_height = 178, 100

save_frame = True  # save the gaze point area
crop_frame = False  # crop the frame or save the whole frame
save_freq = 4  # freq of saving frames
crop_size = (int(200 * 0.8), int(200 * 0.8))
circle_radius = 10


def draw_circle_on_frame(frame, x, y, radius, color, thickness):
    """
    Draw a circle on a given frame. Ensure coordinates are within frame bounds.
    """
    height, width = frame.shape[:2]
    if 0 <= x < width and 0 <= y < height:
        cv2.circle(frame, (x, y), radius=radius, color=color, thickness=thickness)

train_day_list = day_list[:len(day_list) // 4 * 3]
# test_day_list = day_list[len(day_list) // 4 * 3:]

for filename in train_day_list:
    print("Processing gaze file:", filename)
    file_path = os.path.join(xlsx_dir, filename)
    gaze_df = pd.read_excel(file_path, engine='openpyxl')
    gaze_df = gaze_df[gaze_df['Sensor'].str.contains("Eye Tracker", na=False)]
    
    match = re.search(r'(\d+\.d)(?=\.xlsx$)', filename)
    base_part = match.group(1)
    suffix = "day"
    expected_suffix = f"{base_part[:-1]}{suffix}"
    
    # read the video from video_dir and choose the video that matches the filename
    video_dir = '../Data/Video_data'
    
    for video_filename in os.listdir(video_dir):
        if expected_suffix in video_filename:
            video_path = os.path.join(video_dir, video_filename)
            break
    
    video_path = os.path.join(video_path, 'scenevideo.mp4')
    print("Processing video file:", video_path)
    
    # read the video and extract the frames
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    # for debugging
    if fps == 0:
        fps = 24.954767560648392
    print("FPS:", fps)
    microseconds_per_frame = 1e6 / fps

    # Iterate over the video and eye-tracking data
    frame_number = 0
    while cap.isOpened():
        if frame_number % (int(60*fps)) == 0:
            print('Frame number: ', frame_number)
        ret, frame = cap.read()
        if not ret:
            break

        # Find the corresponding gaze points for the current frame
        frame_time_microseconds = int(frame_number * microseconds_per_frame)
        gaze_points = gaze_df[(gaze_df['Recording timestamp'] >= frame_time_microseconds) &
                                      (gaze_df['Recording timestamp'] <
                                       frame_time_microseconds + int(microseconds_per_frame))]
        # for debugging
        # if frame_number == 3702 or frame_number == 3718:
        #     gaze_points.to_excel('gaze_points_{}.xlsx'.format(frame_number), index=False)
            
        """get the gaze points on the frame"""
        if not gaze_points.empty and frame_number % save_freq == 0:
            avg_x = gaze_points['Gaze point X'].mean()
            avg_y = gaze_points['Gaze point Y'].mean()
            
            if not np.isnan(avg_x) and not np.isnan(avg_y):
                avg_x, avg_y = int(avg_x), int(avg_y)
                top_left = (avg_x - rect_width // 2, avg_y - rect_height // 2)
                bottom_right = (avg_x + rect_width // 2, avg_y + rect_height // 2)

                if save_frame:
                    if crop_frame:
                        # Calculate top-left corner of the crop area
                        start_x = max(avg_x - crop_size[0] // 2, 0)
                        start_y = max(avg_y - crop_size[1] // 2, 0)

                        # Ensure the crop area does not exceed the frame dimensions
                        end_x = min(start_x + crop_size[0], frame.shape[1])
                        end_y = min(start_y + crop_size[1], frame.shape[0])
                        start_x = end_x - crop_size[0]
                        start_y = end_y - crop_size[1]
                        cropped_frame = frame[start_y:end_y, start_x:end_x]

                        circle_x, circle_y = avg_x - start_x, avg_y - start_y
                        draw_circle_on_frame(cropped_frame, circle_x, circle_y, 3, (0, 0, 255), -1)
                        
                    else:
                        cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
                        cv2.circle(frame, (avg_x, avg_y), radius=circle_radius, color=(0, 0, 255),
                                   thickness=-1)
                
                    aoi_counts = {}
                    AOI_hit_state = False
                    aoi_columns = [col for col in gaze_df.columns if col.startswith("AOI hit")]
                
                    # Count the number for each AOI hit column
                    for col in aoi_columns:
                        # Sum up all columns for the current frame's gaze points
                        aoi_counts[col] = gaze_points[col].sum()
                
                    # Find the AOI hit column with the highest count
                    if aoi_counts:
                        max_aoi_hit_col = max(aoi_counts, key=aoi_counts.get)
                        max_count = aoi_counts[max_aoi_hit_col]
    
                    # Proceed only if there is at least one AOI hit (max_count > 0)
                    if max_count > 0:
                        # Extract descriptive part from the column name of the most frequent AOI hit
                        aoi_description = max_aoi_hit_col.split(' - ')[-1].rstrip(']')
                        aoi_hit = aoi_description.replace(" ", "").replace(".", "_")  # Convert spaces to underscores for folder name
            
                        output_folder = os.path.join(frame_dir, aoi_hit)
                        os.makedirs(output_folder, exist_ok=True)
                        AOI_hit_state = True
                
                        # cv2.imwrite(os.path.join(output_folder, f'frame_{frame_number}.png'), frame)
                        cv2.imwrite(os.path.join(output_folder, f"{expected_suffix}_frame_{frame_number}.png"), frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_number += 1

    cap.release()
        

Processing gaze file: WaterStreet_DayNight 51081.d.xlsx
Processing video file: ../Data/Video_data/23-12-07_093831_51081.day/scenevideo.mp4
FPS: 24.954769723993763
Frame number:  0
Frame number:  1497
Frame number:  2994
Frame number:  4491
Frame number:  5988
Frame number:  7485
Frame number:  8982
Frame number:  10479
Frame number:  11976
Frame number:  13473
Frame number:  14970
Frame number:  16467
Frame number:  17964
Processing gaze file: WaterStreet_DayNight 93228.d.xlsx
Processing video file: ../Data/Video_data/23-12-20_121322_93228.day/scenevideo.mp4
FPS: 24.954775240240714
Frame number:  0
Frame number:  1497
Frame number:  2994
Frame number:  4491
Frame number:  5988
Frame number:  7485
Frame number:  8982
Frame number:  10479
Frame number:  11976
Frame number:  13473
Frame number:  14970
Frame number:  16467
Processing gaze file: WaterStreet_DayNight 91239.d.xlsx
Processing video file: ../Data/Video_data/23-12-13_121412_91239.day/scenevideo.mp4
FPS: 24.95477038965186
Frame n